In [6]:
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold

# Define constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
K_FOLDS = 5

# Step 1: Create DataFrame
def create_dataframe_from_images(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            label = "post_disaster" if "post_disaster" in filename else "pre_disaster"
            data.append({
                "image_path": os.path.join(directory, filename),
                "label": label
            })
    df = pd.DataFrame(data)
    return df

train_dir = r'C:\Users\Smilika\Documents\Minor_Project\hold_extracted\geotiffs\hold\images'
df = create_dataframe_from_images(train_dir)

# Step 2: Load and preprocess images
def preprocess_image(image_path, label):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

def load_dataset(df):
    images = []
    labels = []
    for _, row in df.iterrows():
        image, label = preprocess_image(row['image_path'], row['label'])
        images.append(image)
        labels.append(0 if label == 'pre_disaster' else 1)  # Binary classification: 0 = pre_disaster, 1 = post_disaster
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_dataset(df)

# Step 3: Define K-Fold Cross-Validation
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

def create_disaster_status_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)  # Batch Normalization
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)  # Dropout layer
    x = BatchNormalization()(x)  # Batch Normalization
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)  # Dropout layer
    x = BatchNormalization()(x)  # Batch Normalization
    predictions = Dense(2, activation='softmax')(x)  # Binary classification: Pre or Post
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_model(X_train, y_train, X_val, y_val):
    model = create_disaster_status_model()
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    return model

# Step 4: K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(images)):
    print(f"Fold {fold + 1}")
    X_train, X_val = images[train_index], images[val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    
    # Train and evaluate model
    model = train_and_evaluate_model(X_train, y_train, X_val, y_val)

Fold 1
Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 32s 501ms/step - accuracy: 0.5931 - loss: 0.8295 - val_accuracy: 0.6684 - val_loss: 0.6075
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 420ms/step - accuracy: 0.7177 - loss: 0.5572 - val_accuracy: 0.7139 - val_loss: 0.5595
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 428ms/step - accuracy: 0.7357 - loss: 0.5222 - val_accuracy: 0.7193 - val_loss: 0.5626
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 421ms/step - accuracy: 0.8093 - loss: 0.4409 - val_accuracy: 0.7273 - val_loss: 0.5509
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 422ms/step - accuracy: 0.8043 - loss: 0.4162 - val_accuracy: 0.7433 - val_loss: 0.5641
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 452ms/step - accuracy: 0.8289 - loss: 0.3678 - val_accuracy: 0.7460 - val_loss: 0.6009
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 421ms/step - accuracy: 0.8487 - loss: 0.3367 - val_accuracy: 0.7273 - val_loss: 0.6197
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 427ms/step - accuracy: 0.8840 - loss: 0.2710 - v

In [44]:
# Step 4: Predict disaster status
def preprocess_image_for_prediction(image_path):
    image, _ = preprocess_image(image_path, None)
    return np.expand_dims(image, axis=0)

def predict_disaster_status(image_path, model):
    image = preprocess_image_for_prediction(image_path)
    prediction = model.predict(image)
    status = np.argmax(prediction, axis=1)[0]
    return "Post-disaster" if status == 1 else "Pre-disaster"

# Example Usage
test_image_path1 = r'C:\Users\Smilika\Documents\Minor_Project\test_extracted\geotiffs\test\images\hurricane-matthew_00000383_post_disaster.png'
test_image_path2 = r'C:\Users\Smilika\Documents\Minor_Project\test_extracted\geotiffs\test\images\hurricane-matthew_00000383_pre_disaster.png'
status = predict_disaster_status(test_image_path, status_model)
print(f"Status: {status}")
status2 = predict_disaster_status(test_image_path2, status_model)
print(f"Status: {status2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Status: Post-disaster
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Status: Pre-disaster


In [45]:
model.save('k_fold_mobilenetV2_damange_disast.h5')